In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "1"


from torchvision.utils import make_grid, save_image
import torchvision.models as models
import torch.nn.functional as F
import matplotlib.pyplot as plt
from copy import deepcopy
import numpy as np
import random
import torch
import json
import PIL


from handler import geozoom_handler
# from helpers import *
from fc_net import *
from utils import *
from sa import *

In [2]:
# random.sample(range(0, len(image_names)), 5)

In [3]:
# image_names = get_png_names("../pooling/data/MEX2/")#[100:105]
image_names = get_png_names("../pooling/data/MEX2/")
image_indices = random.sample(range(0, len(image_names)), 1000)
image_names = [image_names[i] for i in range(len(image_names)) if i in image_indices]
y = get_migrants("../pooling/data/migration_data.json" , image_names)

1000 municipalities.


In [4]:
# for i in image_names:
#     print(load_inputs(i).shape)

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet18 = models.resnet18().to(device)
attn_model = attnNet(resnet = resnet18).to(device)
lr = .0001
criterion = torch.nn.L1Loss(reduction = 'mean')
attn_optimizer = torch.optim.Adam(attn_model.parameters(), lr = lr)

fc_model = fc_net(resnet = resnet18).to(device)
fc_optimizer = torch.optim.Adam(fc_model.parameters(), lr = .01)

butler = geozoom_handler(attn_model, 
                         fc_model, device, 
                         criterion, 
                         attn_optimizer, 
                         fc_optimizer, 
                         num_thresholds = 10,
                         reduction_percent = .90,
                         convergence_dims = (358, 284),
                         plot = False, 
                         v = False)

In [6]:
attn_model.sa

SelfAttention(
  (query_conv): Conv2d(256, 32, kernel_size=(1, 1), stride=(1, 1))
  (key_conv): Conv2d(256, 32, kernel_size=(1, 1), stride=(1, 1))
  (value_conv): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
  (softmax): Softmax(dim=-1)
)

In [7]:
BATCH_SIZE = 1
SPLIT = .60

train_dl, val_dl = butler.prep_attn_data(image_names, y, SPLIT, BATCH_SIZE)

In [8]:
# for i,o in val_dl:
#     print(load_inputs(i[0]).shape, o)

In [ ]:
butler.train_attn_model(train_dl, val_dl)

Epoch:  0
  Training Loss:  1546.095612830321
  Validation Loss:  1810.2818915796279
Loss thresholds for training:  [0.0, 181.0281891579628, 362.0563783159256, 543.0845674738885, 724.1127566318512, 905.1409457898139, 1086.169134947777, 1267.1973241057397, 1448.2255132637024, 1629.2537024216651]
Starting from threshold:  9  with value:  1629.2537024216651


Epoch:  1
  Training Loss:  1514.9631309318543
  Validation Loss:  1788.1201807022094


Epoch:  2
  Training Loss:  1490.8418667348226
  Validation Loss:  1745.2964589118958


Epoch:  3
  Training Loss:  1474.1223415120442
  Validation Loss:  1692.3441275215148


Epoch:  4
  Training Loss:  1463.1278290812174
  Validation Loss:  1702.7102558135987


Epoch:  5
  Training Loss:  1451.9288108317057
  Validation Loss:  1701.4822577667237


Epoch:  6
  Training Loss:  1445.3556198120118
  Validation Loss:  1705.7213681411743


Epoch:  7
  Training Loss:  1437.6554746246338
  Validation Loss:  1695.939594230652


Epoch:  8
  Training Loss:

/opt/conda/envs/rapids/lib/python3.7/site-packages/torch/nn/functional.py:3328: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")




Epoch:  23
  Training Loss:  1371.32966120402
  Validation Loss:  1663.2698483276367


Epoch:  24
  Training Loss:  1367.5335594177245
  Validation Loss:  1648.7203842926026


Epoch:  25
  Training Loss:  1368.045519205729
  Validation Loss:  1652.5007053375243


Epoch:  26
  Training Loss:  1362.9273852030435
  Validation Loss:  1660.7656249237061


Epoch:  27
  Training Loss:  1363.3640367635091
  Validation Loss:  1629.2765606689454


Epoch:  28
  Training Loss:  1361.2716336568196
  Validation Loss:  1629.1995183563233


Epoch:  29
  Training Loss:  1355.0506507873536
  Validation Loss:  1625.2450507354736


Epoch:  30
  Training Loss:  1352.7777104187012
  Validation Loss:  1663.035464401245


Epoch:  31
  Training Loss:  1352.802134806315
  Validation Loss:  1642.6272906494141


Epoch:  32
  Training Loss:  1354.1142830403646
  Validation Loss:  1643.9824560546874


Epoch:  33
  Training Loss:  1353.1140079243978
  Validation Loss:  1626.5539530181884


Epoch:  34
  Training Lo